In [1]:
import json
import pandas as pd
import requests
import datetime
from binance import Client
#import binance_keys
import matplotlib.pyplot as plt
binance_client = Client()
from numba import jit
import numpy as np

Пул ETH USDT, Беру данные с начала года

# Funding rates

In [2]:
start_time = (pd.to_datetime('2022-01-01')- pd.Timestamp("1970-01-01"))// pd.Timedelta('1s')
end_time = (pd.to_datetime('2022-12-05')- pd.Timestamp("1970-01-01"))// pd.Timedelta('1s')
symbol = "ETHUSDT"

In [3]:
#binance Funding
binance_endpoint = 'https://fapi.binance.com'
binance_request = requests.get(binance_endpoint+'/fapi/v1/fundingRate',
                                    {'symbol': symbol,'startTime': start_time*1000,
                                     'endTime': end_time*1000,'limit':1000}).json()
binance_funding = pd.DataFrame(binance_request)
binance_funding["fundingTime"] = [datetime.datetime.fromtimestamp(x / 1000) for x in binance_funding["fundingTime"]]
binance_funding["DateTime"] = [x.replace(microsecond=0) for x in binance_funding['fundingTime']]
binance_funding["fundingRate"] = [float(x) for x in binance_funding['fundingRate']]
binance_funding = binance_funding.drop(["symbol", "fundingTime"], axis = 1)
binance_funding = binance_funding.set_index("DateTime")

# OHLC Data

## Uniswap

In [4]:
query = """{
    pools(
        orderBy: volumeUSD,
        orderDirection: desc,
        first: 1000,
      where: {
       token0: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
       token1: "0xdac17f958d2ee523a2206206994597c13d831ec7"},
        ) {
            id
            token0 {
                symbol
            }
            token1 {
                symbol
            }
            poolHourData(
                orderBy: periodStartUnix
                orderDirection: desc
                ){
                    token0Price
                    token1Price
                    high
                    low
                    open
                    close
      							volumeUSD
      							volumeToken0
      							volumeToken1
      							tvlUSD
      							periodStartUnix
                                feesUSD
                }
            }
        }"""

In [5]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
response = requests.post(url, json={'query': query})
json_data = json.loads(response.text)
a = json_data.get("data").get("pools")[0].get("poolHourData")
_df = pd.DataFrame(a)
_df['date'] = [datetime.datetime.fromtimestamp(x) for x in _df['periodStartUnix']]

* Не получается взять 8 часовые данные на Uniswap, доступны только 3 дня
* Придется брать OHLC с Binance

## Binance костыль

In [6]:
# Binance Spot
def get_binance(symbol, interval, startTime, endTime, limit):
    url = "https://api.binance.com/api/v3/klines"

    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))


    req_params = {"symbol" : symbol, "interval" : interval, "startTime" : startTime, "endTime" : endTime, "limit" : limit}

    _df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text))
    if (len(_df) == 0):
        return None
    
    _df = _df.iloc[:, 0:6]
    _df.columns = ["DateTime", "open", "high", "low", "close", "volume"]
    _df["DateTime"] = [datetime.datetime.fromtimestamp(x / 1000.0) for x in _df["DateTime"]]
    _df = _df.set_index(_df["DateTime"])
    _df = _df.drop(["DateTime","volume"],axis = 1)
    
    
    for i in _df.columns:
        if i == 'date':
            continue
        else:
            _df[i] = [float(x) for x in _df[i]]
    return _df



In [7]:
# Binance-spot
df_binance_spot = get_binance(symbol = symbol, interval = "8h",
                 startTime = datetime.datetime.fromtimestamp(start_time),
                 endTime = datetime.datetime.fromtimestamp(end_time),
                              limit = "1000")

* Если костыль останется, убрать переименование колонок

# TVL day data on UniSwap

In [8]:
query = '''{
    pools(
        orderBy: volumeUSD,
        orderDirection: desc,
        first: 1000,
      where: {
       token0: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
       token1: "0xdac17f958d2ee523a2206206994597c13d831ec7"},
        ) {
            id
            token0 {
                symbol
                volumeUSD
            }
            token1 {
                symbol
            }
            poolDayData(
                orderBy: date
                orderDirection: desc
                ){

      							volumeUSD
      							volumeToken0
      							volumeToken1
      							tvlUSD			
      							date
                    feesUSD
      
                }
            }
        }'''

In [9]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
response = requests.post(url, json={'query': query})
json_data = json.loads(response.text)
a = json_data.get("data").get("pools")[0].get("poolDayData")
fees_tvl = pd.DataFrame(a)
fees_tvl['date'] = [datetime.datetime.fromtimestamp(x) for x in fees_tvl['date']]

for i in fees_tvl.columns:
    if i == 'date':
        continue
    else:
        fees_tvl[i] = [float(x) for x in fees_tvl[i]]

fees_tvl = fees_tvl.set_index("date")

* Почему в часовых данных нет TVL
* Первести в apply изменение форматов в цикле
* Uniswap отдал fees только до августа

# Merge

In [10]:
df = pd.merge(df_binance_spot, binance_funding, left_index=True, right_index=True)

In [11]:
df = df.join(fees_tvl,  how = 'left')

In [12]:
df

,open,high,low,close,fundingRate,volumeUSD,volumeToken0,volumeToken1,tvlUSD,feesUSD
DateTime,,,,,,,,,,
2022-01-01 03:00:00,3676.22,3765.27,3676.22,3715.31,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-01 11:00:00,3715.32,3745.36,3673.46,3725.30,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-01 19:00:00,3725.29,3776.45,3718.71,3765.54,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-02 03:00:00,3765.54,3771.00,3717.30,3747.22,0.000039,NaN,NaN,NaN,NaN,NaN
2022-01-02 11:00:00,3747.22,3790.38,3730.33,3778.17,0.000023,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-11-28 19:00:00,1163.50,1184.98,1151.02,1167.77,-0.000123,NaN,NaN,NaN,NaN,NaN
2022-11-29 03:00:00,1167.77,1215.00,1158.27,1207.08,0.000038,3.339090e+07,27711.964650,3.340869e+07,3.533683e+07,16695.451871
2022-11-29 11:00:00,1207.08,1223.00,1202.96,1208.72,0.000029,NaN,NaN,NaN,NaN,NaN


In [13]:
# Проверочка
df.loc["2022-11-29 03:00:00"]['feesUSD']/df.loc["2022-11-29 03:00:00"]['volumeUSD']

0.0005

### Проблемы
* Uni не возвращает информацию по часам
* По дням получилось запросить только небольшой промежуток с августа

## Заполняем пропуски

* везде интерполируем

In [14]:
def interpolation(columns:list, data_frame):
    for column in columns:
        _ts = data_frame[column]
        for i in range(0, len(_ts), 3):
            try:
                first = _ts[i]
                forth = _ts[i+3]
                diff = (forth - first)/4

                second = first + diff
                third = second + diff

                _ts[i+1] = second
                _ts[i+2] = third
            except:
                break
        data_frame[column] = _ts
    return data_frame

In [15]:
to_change = ['volumeUSD', 'volumeToken0', 'volumeToken1', 'tvlUSD', 'feesUSD']
interpolation(to_change, df)

,open,high,low,close,fundingRate,volumeUSD,volumeToken0,volumeToken1,tvlUSD,feesUSD
DateTime,,,,,,,,,,
2022-01-01 03:00:00,3676.22,3765.27,3676.22,3715.31,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-01 11:00:00,3715.32,3745.36,3673.46,3725.30,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-01 19:00:00,3725.29,3776.45,3718.71,3765.54,0.000100,NaN,NaN,NaN,NaN,NaN
2022-01-02 03:00:00,3765.54,3771.00,3717.30,3747.22,0.000039,NaN,NaN,NaN,NaN,NaN
2022-01-02 11:00:00,3747.22,3790.38,3730.33,3778.17,0.000023,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-11-28 19:00:00,1163.50,1184.98,1151.02,1167.77,-0.000123,3.666054e+07,30582.235716,3.705335e+07,3.499069e+07,18330.271784
2022-11-29 03:00:00,1167.77,1215.00,1158.27,1207.08,0.000038,3.339090e+07,27711.964650,3.340869e+07,3.533683e+07,16695.451871
2022-11-29 11:00:00,1207.08,1223.00,1202.96,1208.72,0.000029,3.407612e+07,27875.022188,3.408984e+07,3.512609e+07,17038.061967


In [16]:
df = df.dropna()

In [17]:
df

,open,high,low,close,fundingRate,volumeUSD,volumeToken0,volumeToken1,tvlUSD,feesUSD
DateTime,,,,,,,,,,
2022-08-29 03:00:00,1426.76,1463.03,1422.08,1454.74,-0.000248,4.761278e+07,31610.128775,4.766525e+07,3.805135e+07,23806.389268
2022-08-29 11:00:00,1454.74,1531.37,1442.00,1521.89,-0.000171,5.327341e+07,35058.567930,5.331230e+07,3.814201e+07,26636.703141
2022-08-29 19:00:00,1521.88,1559.99,1519.97,1551.80,-0.000067,5.893403e+07,38507.007086,5.895935e+07,3.823268e+07,29467.017014
2022-08-30 03:00:00,1551.79,1600.84,1523.56,1586.98,-0.000095,7.025529e+07,45403.885397,7.025345e+07,3.841401e+07,35127.644761
2022-08-30 11:00:00,1586.98,1606.13,1509.61,1524.69,-0.000067,7.426874e+07,47765.514828,7.425486e+07,3.728333e+07,37134.369488
...,...,...,...,...,...,...,...,...,...,...
2022-11-28 19:00:00,1163.50,1184.98,1151.02,1167.77,-0.000123,3.666054e+07,30582.235716,3.705335e+07,3.499069e+07,18330.271784
2022-11-29 03:00:00,1167.77,1215.00,1158.27,1207.08,0.000038,3.339090e+07,27711.964650,3.340869e+07,3.533683e+07,16695.451871
2022-11-29 11:00:00,1207.08,1223.00,1202.96,1208.72,0.000029,3.407612e+07,27875.022188,3.408984e+07,3.512609e+07,17038.061967


### Проблемы 
* Наверное стоит из изменений количества токенов посчитать volume USD
* Как получить нормальный TVL


* Заполняем TVL через интерполяцию
* Как посчитать volume token 0
* Считаем K
* Интерполируем объем (Или считаем, как показано ниже)
* Считаем от него объем комиссий

## Predict Диапазон ликвидности

In [18]:
_df = df
_df.index[0]

Timestamp('2022-08-29 03:00:00')

In [19]:
# Get last 24 hours to predict next 8 hour interval
def GetHistoricalData(symbol, interval, fromDate, toDate):
    klines = client.get_historical_klines(symbol, interval, fromDate, toDate)
    df = pd.DataFrame(klines, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    df.dateTime = pd.to_datetime(df.dateTime, unit='ms')
    df['date'] = df.dateTime.dt.strftime("%d/%m/%Y")
    df['time'] = df.dateTime.dt.strftime("%H:%M:%S")
    df = df.drop(['dateTime', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)
    column_names = ["date", "time", "open", "high", "low", "volume"]
    df = df.reindex(columns=column_names)
    return df
    
api_key = ""
api_secret = ""
client = Client(api_key, api_secret)

#########
fromDate = (_df.index[0] + datetime.timedelta(days=-1)).strftime("%m/%d/%Y, %H:%M:%S")
toDate = _df.index[0].strftime("%m/%d/%Y, %H:%M:%S")
##########

symbol = "ETHUSDT"
interval = Client.KLINE_INTERVAL_5MINUTE # луч

In [20]:
def make_new_col(df):
  
    mass = np.zeros(len(df))
    mass[0] = 1 # первую точку доопределяем 1
    values = df['open'].values.astype(float)
    for i in range(1, len(df)):
        mass[i] = values[i]/values[i-1]
    df['change'] = mass
    return df

In [21]:
# Более быстрая реализация (здесь нет циктов и используется компиляция jit)
@jit
def bootstrap(mass, n_forcast, intervals_in_hour):
    '''Функция бутстрэпит изменения за прошедшие часы (это длина массива) n_forcast*intervals_in_hour раз, 
    процедура повторяется 10000 раз, в резульатате мы перемножеаем значения и выдаем прогноз в каждой точке таким образом (10000 симуляций)'''
    mass_res = np.zeros((10000, 2)) # 2 - нижняя и верхняя границы для каждой симуляции
    mass = np.array(mass) # переводим в numpy
    n = len(mass) # длина выборки для обучения
    size_forecast = n_forcast*intervals_in_hour
    # случайно берем числа до n, размера n_forcast*intervals_in_hour
    indexes = np.random.randint(n, size=size_forecast*10000)
    change = mass[[indexes]]
    change = change.reshape((10000, size_forecast))
    # теперь рассомтрим куммулятивное перемножение- предсказанное поведение
    cum_change = np.cumprod(change, axis=1)
    # минимальное и максимальные значения внутри временного интервалла
    min_ = np.min(cum_change, axis=1)
    max_ = np.max(cum_change, axis=1)
    sorted_res_min = np.sort(min_)
    sorted_res_max = np.sort(max_)
    high, low = sorted_res_max[9500], sorted_res_min[500]
    return np.array([high, low])

In [22]:
#Получаем данные
df_current = GetHistoricalData(symbol, interval, fromDate, toDate)
# Добавляем новую колонку
df_current = make_new_col(df_current)
# требуется для абсолютных значений запомнить текущую
current_value = df_current['open'].values.astype(float)[-1]
# Рассчитываем сам интервал
res_interval = bootstrap(df_current['change'].values, n_forcast=8, intervals_in_hour=12)*current_value

/var/folders/hd/6964l85j4gx1hgy42r98qngc0000gp/T/ipykernel_17103/1757968330.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "bootstrap" failed type inference due to: No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
  - Of which 4 did not match due to:
  Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
    With argument(s): '(array(float64, 1d, C))':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   No implementation of function Function(<intrinsic stub>) found for signature:
    
    >>> stub(array(float64, 1d, C))
    
   There are 2 candidate implementations:
     - Of which 2 did not match due to:
     Intrinsic in function 'stub': File: numba/core/overload_glu

In [23]:
intervals = []
for i in range(len(df)):
    
    fromDate = (df.index[i] + datetime.timedelta(days=-1)).strftime("%m/%d/%Y, %H:%M:%S")
    toDate = df.index[i].strftime("%m/%d/%Y, %H:%M:%S")
    
    #Получаем данные
    df_current = GetHistoricalData(symbol, interval, fromDate, toDate)
    # Добавляем новую колонку
    df_current = make_new_col(df_current)
    # требуется для абсолютных значений запомнить текущую
    current_value = df_current['open'].values.astype(float)[-1]
    # Рассчитываем сам интервал
    res_interval = bootstrap(df_current['change'].values, n_forcast=8, intervals_in_hour=12)*current_value
    intervals.append(list(res_interval))
    


/var/folders/hd/6964l85j4gx1hgy42r98qngc0000gp/T/ipykernel_17103/3863686115.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res_interval = bootstrap(df_current['change'].values, n_forcast=8, intervals_in_hour=12)*current_value
/var/folders/hd/6964l85j4gx1hgy42r98qngc0000gp/T/ipykernel_17103/3863686115.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res_interval = bootstrap(df_current['change'].values, n_forcast=8, intervals_in_hour=12)*current_value
/var/folders/hd/6964l85j4gx1hgy42r98qngc0000gp/T/ipykernel_17103/3863686115.py:1

In [24]:
pred_inter = pd.DataFrame(intervals)
pred_inter.columns = ["Pred_high", "Pred_low"]
pred_inter.index  = df.index

In [25]:
df = df.join(pred_inter,  how = 'left')

In [26]:
df.head()

,open,high,low,close,fundingRate,volumeUSD,volumeToken0,volumeToken1,tvlUSD,feesUSD,Pred_high,Pred_low
DateTime,,,,,,,,,,,,
2022-08-29 03:00:00,1426.76,1463.03,1422.08,1454.74,-0.000248,4.761278e+07,31610.128775,4.766525e+07,3.805135e+07,23806.389268,1487.507849,1382.098822
2022-08-29 11:00:00,1454.74,1531.37,1442.00,1521.89,-0.000171,5.327341e+07,35058.567930,5.331230e+07,3.814201e+07,26636.703141,1495.058483,1389.104384
2022-08-29 19:00:00,1521.88,1559.99,1519.97,1551.80,-0.000067,5.893403e+07,38507.007086,5.895935e+07,3.823268e+07,29467.017014,1620.488758,1475.806218
2022-08-30 03:00:00,1551.79,1600.84,1523.56,1586.98,-0.000095,7.025529e+07,45403.885397,7.025345e+07,3.841401e+07,35127.644761,1639.554627,1494.979605
2022-08-30 11:00:00,1586.98,1606.13,1509.61,1524.69,-0.000067,7.426874e+07,47765.514828,7.425486e+07,3.728333e+07,37134.369488,1710.507456,1546.866371


# Calculate Hedge

In [36]:
hedge_list = []
for i in range(len(df)):
    r = np.sqrt(df["Pred_high"][i]/df["Pred_low"][i]) 
    h = (np.sqrt(r)-1)/(r-1)
    hedge_list.append(h)    

In [38]:
df["Hedge"] = hedge_list

In [39]:
df.head()

,open,high,low,close,fundingRate,volumeUSD,volumeToken0,volumeToken1,tvlUSD,feesUSD,Pred_high,Pred_low,Hedge
DateTime,,,,,,,,,,,,,
2022-08-29 03:00:00,1426.76,1463.03,1422.08,1454.74,-0.000248,4.761278e+07,31610.128775,4.766525e+07,3.805135e+07,23806.389268,1487.507849,1382.098822,0.495406
2022-08-29 11:00:00,1454.74,1531.37,1442.00,1521.89,-0.000171,5.327341e+07,35058.567930,5.331230e+07,3.814201e+07,26636.703141,1495.058483,1389.104384,0.495406
2022-08-29 19:00:00,1521.88,1559.99,1519.97,1551.80,-0.000067,5.893403e+07,38507.007086,5.895935e+07,3.823268e+07,29467.017014,1620.488758,1475.806218,0.494155
2022-08-30 03:00:00,1551.79,1600.84,1523.56,1586.98,-0.000095,7.025529e+07,45403.885397,7.025345e+07,3.841401e+07,35127.644761,1639.554627,1494.979605,0.494231
2022-08-30 11:00:00,1586.98,1606.13,1509.61,1524.69,-0.000067,7.426874e+07,47765.514828,7.425486e+07,3.728333e+07,37134.369488,1710.507456,1546.866371,0.493715


# Download gas fees

* Откуда их тащить?
* Будем считать их == 5 долларам за каждое извлечение ликвидности

# Calculate Position Value

* Мы предполагаем, что в пуле налиты равными долями токены
* Как достать соотношение токенов в пуле, если Volume - это не то, что нужно

In [42]:
money = 10000 # We Have 10 000 USDT

In [47]:
4.761278e+07 / 4.766525e+07

0.998899198053089

In [43]:
open_price = 1426.76
n_alt = money/((2+h)*open_price)
np.round(n_alt,4) # округляем до 4 знака после запятой

2.809507040028925

In [46]:
31610.128775  * 1426.76  / 4.761278e+07

0.9472260878490817

In [48]:
_df = df[["open", 'volumeToken0', 'volumeToken1']]
proportion = []


In [50]:
_df['proportion'] = _df['volumeToken0'] * _df['open'] / _df['volumeToken1']

/var/folders/hd/6964l85j4gx1hgy42r98qngc0000gp/T/ipykernel_17103/1659902352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['proportion'] = _df['volumeToken0'] * _df['open'] / _df['volumeToken1']


In [52]:
_df

,open,volumeToken0,volumeToken1,proportion
DateTime,,,,
2022-08-29 03:00:00,1426.76,31610.128775,4.766525e+07,0.946183
2022-08-29 11:00:00,1454.74,35058.567930,5.331230e+07,0.956648
2022-08-29 19:00:00,1521.88,38507.007086,5.895935e+07,0.993957
2022-08-30 03:00:00,1551.79,45403.885397,7.025345e+07,1.002902
2022-08-30 11:00:00,1586.98,47765.514828,7.425486e+07,1.020848
...,...,...,...,...
2022-11-28 19:00:00,1163.50,30582.235716,3.705335e+07,0.960303
2022-11-29 03:00:00,1167.77,27711.964650,3.340869e+07,0.968646
2022-11-29 11:00:00,1207.08,27875.022188,3.408984e+07,0.987021


# Calculate Collected Fees

In [53]:
3.449389e+07

34493890.0

# Calculate PnL